In [1]:
import numpy as np
import pandas as pd
import random

from pathlib import Path
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras.metrics import AUC
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, GRU, Input, BatchNormalization, Dropout, TimeDistributed
from ncps.wirings import AutoNCP
from ncps.keras import LTC

2025-03-22 00:03:25.943562: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-22 00:03:25.960151: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-22 00:03:25.989487: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742591006.017360 1358316 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742591006.025730 1358316 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-22 00:03:26.077007: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

# Configuration

In [2]:
NUM_EPOCHS = 200
NUM_EXPERIMENTS = 5

def create_model(train):
    model = Sequential()
    model.add(Input(shape=(train.shape[1], train.shape[2])))

    model.add(Dropout(0.2))
    model.add(LTC(10, return_sequences=True))

    model.add(Dropout(0.2))
    model.add(LTC(10, return_sequences=True))

    model.add(Dropout(0.2))
    model.add(LTC(10, return_sequences=False))

    model.add(Dropout(0.2))
    model.add(Dense(6, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=Adam(learning_rate=0.003), loss='binary_crossentropy', metrics=["accuracy", AUC(name="auc")])
    return model

# Experiment

In [ ]:
ID = ["ID"]
USER = ["SubjectID"]
IDS = ["SubjectID", "VideoID"]
TARGET = ["predefinedlabel"]
FEATURES = ["Raw", "Delta", "Theta", "Alpha1", "Alpha2", "Beta1", "Beta2", "Gamma1", "Gamma2"]
LAGS = [1]
INIT_SEED = 5412

In [4]:
data_dir = Path("/home/aseliverstov/projects/brain_signals/data")
data = pd.read_csv(data_dir / "EEG_data.csv")

new_features = []
for lag in LAGS:
    for feature_name in FEATURES:
        new_feature_name = f"{feature_name}_{lag}"
        new_features.append(new_feature_name)
        data[new_feature_name] = data.groupby(IDS)[feature_name].shift(lag).fillna(0)
FEATURES.extend(new_features)

data["ID"] = (len(np.unique(data["VideoID"])) * data["SubjectID"] + data["VideoID"]).astype("int")
data = data[ID + USER + FEATURES + TARGET]

data.head(3)

,ID,SubjectID,Raw,Delta,Theta,Alpha1,Alpha2,Beta1,Beta2,Gamma1,...,Raw_1,Delta_1,Theta_1,Alpha1_1,Alpha2_1,Beta1_1,Beta2_1,Gamma1_1,Gamma2_1,predefinedlabel
0,0,0.0,278.0,301963.0,90612.0,33735.0,23991.0,27946.0,45097.0,33228.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0.0,-50.0,73787.0,28083.0,1439.0,2240.0,2746.0,3687.0,5293.0,...,278.0,301963.0,90612.0,33735.0,23991.0,27946.0,45097.0,33228.0,8293.0,0.0
2,0,0.0,101.0,758353.0,383745.0,201999.0,62107.0,36293.0,130536.0,57243.0,...,-50.0,73787.0,28083.0,1439.0,2240.0,2746.0,3687.0,5293.0,2740.0,0.0


In [5]:
def reshape_dataset(data):
    features = []
    target = []
    for cur_id in np.unique(data[ID].to_numpy()):
        cur_id_data = data[data[ID].to_numpy() == cur_id]
        target.append(np.mean(cur_id_data[TARGET].to_numpy()).astype("int"))
        features.append(cur_id_data[FEATURES].to_numpy())

    features = pad_sequences(features)
    return np.array(features), np.array(target)

def pad_sequences(arrays, pad_value=0):
    max_length = max(arr.shape[0] for arr in arrays)
    padded_arrays = [
        np.pad(
            arr,
            ((0, max_length - arr.shape[0]), (0, 0)),
            mode='constant',
            constant_values=pad_value)
            for arr in arrays
        ]
    return np.stack(padded_arrays)

In [6]:
X, _ = reshape_dataset(data)
model = create_model(X)
model.summary()

2025-03-22 00:03:29.443542: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dropout (Dropout)               │ (None, 144, 18)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ltc (LTC)                       │ (None, 144, 10)        │         1,206 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 144, 10)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ltc_1 (LTC)                     │ (None, 144, 10)        │           870 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 144, 10)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ltc_2 (LTC)                     │ (None, 10)             │           870 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 10)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 6)              │            66 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │             7 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,019 (11.79 KB)

 Trainable params: 3,019 (11.79 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
%%time

all_acc = []
all_loss = []
all_auc = []

all_val_acc = []
all_val_loss = []
all_val_auc = []

for j, seed in tqdm(enumerate(np.arange(NUM_EXPERIMENTS) + INIT_SEED)):
    np.random.seed(int(seed))
    random.seed(int(seed))
    tf.random.set_seed(int(seed))

    train_id = np.random.choice(np.unique(np.ravel(data[USER])), 7, replace=False)
    train_index = np.isin(data[USER], train_id)

    train = data.iloc[train_index]
    test = data.iloc[~train_index]

    X_train, y_train = reshape_dataset(train)
    X_test, y_test = reshape_dataset(test)

    y_train = y_train.reshape(-1, 1)
    y_test = y_test.reshape(-1, 1)

    model = create_model(X_train)

    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=NUM_EPOCHS,
        batch_size=10,
        verbose=1,
    )

    acc = history.history['accuracy']
    loss = history.history['loss']
    auc = history.history['auc']

    val_acc = history.history['val_accuracy']
    val_loss = history.history['val_loss']
    val_auc = history.history['val_auc']

    all_acc.append(acc)
    all_loss.append(loss)
    all_auc.append(auc)

    all_val_acc.append(val_acc)
    all_val_loss.append(val_loss)
    all_val_auc.append(val_auc)

epoch_acc = np.mean(all_acc, axis=0)
epoch_loss = np.mean(all_loss, axis=0)
epoch_auc = np.mean(all_auc, axis=0)

epoch_val_acc = np.mean(all_val_acc, axis=0)
epoch_val_loss = np.mean(all_val_loss, axis=0)
epoch_val_auc = np.mean(all_val_auc, axis=0)

0it [00:00, ?it/s]

Epoch 1/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 16s 844ms/step - accuracy: 0.5471 - auc: 0.6494 - loss: 0.6916 - val_accuracy: 0.5000 - val_auc: 0.6333 - val_loss: 0.6959
Epoch 2/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 739ms/step - accuracy: 0.4160 - auc: 0.5337 - loss: 0.7033 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.6934
Epoch 3/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 567ms/step - accuracy: 0.5258 - auc: 0.4990 - loss: 0.6956 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.6932
Epoch 4/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 572ms/step - accuracy: 0.4188 - auc: 0.4240 - loss: 0.7111 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.6932
Epoch 5/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 959ms/step - accuracy: 0.4200 - auc: 0.4179 - loss: 0.7015 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.6933
Epoch 6/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 681ms/step - accuracy: 0.4549 - auc: 0.4358 - loss: 0.7105 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.6933
Epoch 7/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 825ms

1it [24:46, 1486.33s/it]

Epoch 1/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 35s 1s/step - accuracy: 0.5668 - auc: 0.6065 - loss: 0.6795 - val_accuracy: 0.5000 - val_auc: 0.6000 - val_loss: 0.6958
Epoch 2/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 778ms/step - accuracy: 0.5753 - auc: 0.5773 - loss: 0.6775 - val_accuracy: 0.6000 - val_auc: 0.6000 - val_loss: 0.6875
Epoch 3/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 794ms/step - accuracy: 0.5689 - auc: 0.5967 - loss: 0.6751 - val_accuracy: 0.6000 - val_auc: 0.6000 - val_loss: 0.6910
Epoch 4/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 11s 885ms/step - accuracy: 0.4177 - auc: 0.3689 - loss: 0.7085 - val_accuracy: 0.6000 - val_auc: 0.6000 - val_loss: 0.6870
Epoch 5/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 838ms/step - accuracy: 0.6362 - auc: 0.6132 - loss: 0.6781 - val_accuracy: 0.5000 - val_auc: 0.6000 - val_loss: 0.6843
Epoch 6/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 891ms/step - accuracy: 0.5374 - auc: 0.4953 - loss: 0.6850 - val_accuracy: 0.5000 - val_auc: 0.7000 - val_loss: 0.6849
Epoch 7/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 970ms/

2it [49:11, 1473.96s/it]

...

Epoch 1/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 37s 1s/step - accuracy: 0.5005 - auc: 0.4736 - loss: 0.7141 - val_accuracy: 0.5000 - val_auc: 0.7333 - val_loss: 0.6932
Epoch 2/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 824ms/step - accuracy: 0.3886 - auc: 0.3238 - loss: 0.7357 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.6954
Epoch 3/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 834ms/step - accuracy: 0.5269 - auc: 0.5520 - loss: 0.6891 - val_accuracy: 0.5000 - val_auc: 0.6333 - val_loss: 0.6926
Epoch 4/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 851ms/step - accuracy: 0.5307 - auc: 0.4960 - loss: 0.6955 - val_accuracy: 0.5000 - val_auc: 0.6333 - val_loss: 0.6923
Epoch 5/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 11s 908ms/step - accuracy: 0.4584 - auc: 0.4786 - loss: 0.6987 - val_accuracy: 0.5000 - val_auc: 0.6333 - val_loss: 0.6919
Epoch 6/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 820ms/step - accuracy: 0.5841 - auc: 0.5432 - loss: 0.6903 - val_accuracy: 0.5000 - val_auc: 0.6333 - val_loss: 0.6895
Epoch 7/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/ste

4it [1:39:53, 1506.63s/it]

Epoch 1/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 37s 1s/step - accuracy: 0.4897 - auc: 0.4537 - loss: 0.6978 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.6931
Epoch 2/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 873ms/step - accuracy: 0.5368 - auc: 0.5307 - loss: 0.6791 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.6932
Epoch 3/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 824ms/step - accuracy: 0.6392 - auc: 0.5362 - loss: 0.6911 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.6932
Epoch 4/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 11s 869ms/step - accuracy: 0.3992 - auc: 0.5457 - loss: 0.6911 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.6932
Epoch 5/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 863ms/step - accuracy: 0.5004 - auc: 0.5978 - loss: 0.6948 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.6932
Epoch 6/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 791ms/step - accuracy: 0.4720 - auc: 0.4432 - loss: 0.6964 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.6931
Epoch 7/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 903ms/s

5it [2:05:43, 1508.65s/it]

CPU times: user 3h 35min 27s, sys: 1h 1min 28s, total: 4h 36min 55s
Wall time: 2h 5min 43s


In [ ]:
for i in range(NUM_EPOCHS):
    print(f"Epoch {(i + 1)}: TRAIN Accuracy = {np.round(epoch_acc[i], 3)} Loss = {np.round(epoch_loss[i], 3)} AUC = {np.round(epoch_auc[i], 3)}")
    print(f"Epoch {(i + 1)}: VAL Accuracy = {np.round(epoch_val_acc[i], 3)} Loss = {np.round(epoch_val_loss[i], 3)} AUC = {np.round(epoch_val_auc[i], 3)}")

Epoch 1: TRAIN Accuracy = 0.503 Loss = 0.702 AUC = 0.537
Epoch 1: VAL Accuracy = 0.5 Loss = 0.696 AUC = 0.62
Epoch 2: TRAIN Accuracy = 0.46 Loss = 0.703 AUC = 0.48
Epoch 2: VAL Accuracy = 0.52 Loss = 0.692 AUC = 0.547
Epoch 3: TRAIN Accuracy = 0.546 Loss = 0.691 AUC = 0.545
Epoch 3: VAL Accuracy = 0.52 Loss = 0.692 AUC = 0.573
Epoch 4: TRAIN Accuracy = 0.471 Loss = 0.696 AUC = 0.484
Epoch 4: VAL Accuracy = 0.52 Loss = 0.692 AUC = 0.573
Epoch 5: TRAIN Accuracy = 0.511 Loss = 0.695 AUC = 0.516
Epoch 5: VAL Accuracy = 0.5 Loss = 0.691 AUC = 0.573
Epoch 6: TRAIN Accuracy = 0.483 Loss = 0.698 AUC = 0.478
Epoch 6: VAL Accuracy = 0.5 Loss = 0.691 AUC = 0.593
Epoch 7: TRAIN Accuracy = 0.554 Loss = 0.693 AUC = 0.546
Epoch 7: VAL Accuracy = 0.527 Loss = 0.689 AUC = 0.593
Epoch 8: TRAIN Accuracy = 0.517 Loss = 0.693 AUC = 0.513
Epoch 8: VAL Accuracy = 0.547 Loss = 0.689 AUC = 0.58
Epoch 9: TRAIN Accuracy = 0.554 Loss = 0.687 AUC = 0.552
Epoch 9: VAL Accuracy = 0.547 Loss = 0.688 AUC = 0.547
Epoch